# 경사하강법

https://gjustin40.github.io/pytorch/2020/12/13/Pytorch-GradientDescent.html

- 앞, 뒤 부분은 모두 생략하고 학습하는 부분만 작성하겠다.

#  Batch Gradient Descent(BGD)

In [ ]:
# 데이터 정의
# 모델 정의
# 손실 및 최적화 함수 정의

for e in range(EPOCH):
    
    train_loss = 0
    for data in dataloader: # batch_size=1인 dataloader
        image, label = data # image = 1개
        
        output = model(image)
        loss = loss_func(output, label)
        
        train_loss = train_loss + loss
        ### error 계산 완료
        
    optimizer.zero_grad()    
    train_loss.backward()
    otimizer.step()   

# Stochastic Gradient Descent(SGD)

In [ ]:
# 데이터 정의
# 모델 정의
# 손실 및 최적화 함수 정의

for e in range(EPOCH):
    
    for data in dataloader: # batch_size=1인 dataloader
        image, label = data # image = 1개
        
        optimizer.zero_grad()
        
        output = model(image)
        loss = loss_func(output, label)
        loss.backward()
        
        optimizer.step()      

# mini-batch Stochastic Gradient Descent(MSGD)

In [ ]:
# 데이터 정의
# 모델 정의
# 손실 및 최적화 함수 정의

BATCH_SIZE = 10
dataloader = dataloader(batch_size = BATCH_SIZE)

for e in range(EPOCH):
    
    for data in dataloader: # batch_size=10인 dataloader
        image, label = data # image = 10개
        
        optimizer.zero_grad()
        
        output = model(image)
        loss = loss_func(output, label)
        loss.backward()
        
        optimizer.step()      

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import numpy as np

In [2]:
# Dataset & DataLoader

transform = transforms.Compose(
[
    transforms.ToTensor(),
    transforms.Normalize([0], [1])
])

trainset = torchvision.datasets.MNIST(root = '../data', train = True,
                                      download = True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size = 16,
                                          shuffle = True, num_workers=1)

testset = torchvision.datasets.MNIST(root = '../data', train = False,
                                     download = True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size = 128,
                                         shuffle = True, num_workers=1)

classes = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}

print(trainset)
print('*'*100)
print(testset)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ../data
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0], std=[1])
           )
****************************************************************************************************
Dataset MNIST
    Number of datapoints: 10000
    Root location: ../data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=[0], std=[1])
           )


In [4]:
# Make Model

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 7, padding = 3)
        self.conv2 = nn.Conv2d(10, 50, 7, padding = 3)
        self.conv3 = nn.Conv2d(50, 120, 7, padding = 3)
        self.conv4 = nn.Conv2d(120, 100, 5)
        self.conv5 = nn.Conv2d(100, 20, 5)
        self.conv6 = nn.Conv2d(20, 10, 5)
        self.conv7 = nn.Conv2d(10, 10, 3)
        self.pool = nn.MaxPool2d(2,2)
        self.fc1 = nn.Linear(10 * 8 * 8, 120)
        self.fc2 = nn.Linear(120, 360)
        self.fc3 = nn.Linear(360, 50)
        self.fc4 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x)) # 28 28
        x = F.relu(self.conv2(x)) # 28 28
        x = F.relu(self.conv3(x)) # 28 28
        x = F.relu(self.conv4(x)) # 24 24
        x = F.relu(self.conv5(x)) # 20 20
        x = self.pool(F.relu(self.conv6(x))) # 8 8
        x = x.view(-1, 10 * 8 * 8)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        
        return x

model = Net()

In [5]:
# GPU 사용여부 판단
if torch.cuda.is_available():
    device = torch.device('cuda')
    print('We can use GPU')
else:
    device = torch.device('cpu')
    print('We can use CPU')

We can use GPU


In [6]:
import torch.optim as optim

model = Net().to(device)

loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)
print(loss_func, optimizer)

CrossEntropyLoss() SGD (
Parameter Group 0
    dampening: 0
    lr: 0.1
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [7]:
# Train

import time

EPOCH = 1

for e in range(1, EPOCH+1):
    model.train()
    start_time = time.time()
    running_loss = 0
    
    for i, data in enumerate(trainloader):
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_func(outputs, labels)
        print('-------------------')
        print('loss : ', loss)
        loss.backward()
        optimizer.step()
        
        running_loss += loss
        now = time.time()
        print('\r[%d/%d]-----[%d/%d] LOSS : %.3f------ Time : %d' 
              %(e, EPOCH, i, 60000/128, running_loss, now - start_time), end = '')
        
    print('\n')

-------------------
loss :  tensor(2.3147, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[0/468] LOSS : 2.315------ Time : 2-------------------
loss :  tensor(2.3191, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[1/468] LOSS : 4.634------ Time : 2-------------------
loss :  tensor(2.3007, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[2/468] LOSS : 6.934------ Time : 2-------------------
loss :  tensor(2.2807, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[3/468] LOSS : 9.215------ Time : 2-------------------
loss :  tensor(2.3111, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[4/468] LOSS : 11.526------ Time : 2-------------------
loss :  tensor(2.3419, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[5/468] LOSS : 13.868------ Time : 2-------------------
loss :  tensor(2.3374, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[6/468] LOSS : 16.206------ Time : 2-------------------
loss :  tensor(2.2979, device='cuda:0', grad_fn=<NllL

[1/1]-----[68/468] LOSS : 158.951------ Time : 3-------------------
loss :  tensor(2.3080, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[69/468] LOSS : 161.259------ Time : 3-------------------
loss :  tensor(2.3116, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[70/468] LOSS : 163.570------ Time : 3-------------------
loss :  tensor(2.2829, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[71/468] LOSS : 165.853------ Time : 3-------------------
loss :  tensor(2.2893, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[72/468] LOSS : 168.142------ Time : 3-------------------
loss :  tensor(2.3111, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[73/468] LOSS : 170.454------ Time : 3-------------------
loss :  tensor(2.3012, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[74/468] LOSS : 172.755------ Time : 3-------------------
loss :  tensor(2.3008, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[75/468] LOSS : 175.056------ Time : 3-------

[1/1]-----[145/468] LOSS : 336.105------ Time : 4-------------------
loss :  tensor(2.3360, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[146/468] LOSS : 338.441------ Time : 4-------------------
loss :  tensor(2.3558, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[147/468] LOSS : 340.796------ Time : 4-------------------
loss :  tensor(2.3048, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[148/468] LOSS : 343.101------ Time : 4-------------------
loss :  tensor(2.3085, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[149/468] LOSS : 345.410------ Time : 4-------------------
loss :  tensor(2.3002, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[150/468] LOSS : 347.710------ Time : 4-------------------
loss :  tensor(2.2981, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[151/468] LOSS : 350.008------ Time : 4-------------------
loss :  tensor(2.3046, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[152/468] LOSS : 352.313------ Time : 

[1/1]-----[223/468] LOSS : 516.100------ Time : 5-------------------
loss :  tensor(2.3089, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[224/468] LOSS : 518.409------ Time : 5-------------------
loss :  tensor(2.3163, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[225/468] LOSS : 520.725------ Time : 5-------------------
loss :  tensor(2.3091, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[226/468] LOSS : 523.034------ Time : 5-------------------
loss :  tensor(2.3120, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[227/468] LOSS : 525.346------ Time : 5-------------------
loss :  tensor(2.2965, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[228/468] LOSS : 527.643------ Time : 5-------------------
loss :  tensor(2.3126, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[229/468] LOSS : 529.955------ Time : 5-------------------
loss :  tensor(2.3005, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[230/468] LOSS : 532.256------ Time : 

[1/1]-----[301/468] LOSS : 695.828------ Time : 5-------------------
loss :  tensor(2.2809, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[302/468] LOSS : 698.109------ Time : 5-------------------
loss :  tensor(2.3080, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[303/468] LOSS : 700.417------ Time : 5-------------------
loss :  tensor(2.3181, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[304/468] LOSS : 702.736------ Time : 5-------------------
loss :  tensor(2.3056, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[305/468] LOSS : 705.041------ Time : 5-------------------
loss :  tensor(2.3190, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[306/468] LOSS : 707.360------ Time : 5-------------------
loss :  tensor(2.3027, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[307/468] LOSS : 709.663------ Time : 5-------------------
loss :  tensor(2.2809, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[308/468] LOSS : 711.944------ Time : 

...

[1/1]-----[615/468] LOSS : 1418.791------ Time : 8-------------------
loss :  tensor(2.2972, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[616/468] LOSS : 1421.088------ Time : 8-------------------
loss :  tensor(2.3130, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[617/468] LOSS : 1423.401------ Time : 8-------------------
loss :  tensor(2.2970, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[618/468] LOSS : 1425.698------ Time : 8-------------------
loss :  tensor(2.3051, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[619/468] LOSS : 1428.003------ Time : 8-------------------
loss :  tensor(2.3042, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[620/468] LOSS : 1430.307------ Time : 8-------------------
loss :  tensor(2.2908, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[621/468] LOSS : 1432.598------ Time : 8-------------------
loss :  tensor(2.3083, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[622/468] LOSS : 1434.906------

loss :  tensor(2.2943, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[694/468] LOSS : 1600.652------ Time : 9-------------------
loss :  tensor(2.3391, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[695/468] LOSS : 1602.991------ Time : 9-------------------
loss :  tensor(2.3233, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[696/468] LOSS : 1605.314------ Time : 9-------------------
loss :  tensor(2.2916, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[697/468] LOSS : 1607.606------ Time : 9-------------------
loss :  tensor(2.2926, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[698/468] LOSS : 1609.898------ Time : 9-------------------
loss :  tensor(2.3092, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[699/468] LOSS : 1612.208------ Time : 9-------------------
loss :  tensor(2.2703, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[700/468] LOSS : 1614.478------ Time : 9-------------------
loss :  tensor(2.3121, device='cuda:0', g

loss :  tensor(2.3402, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[770/468] LOSS : 1775.746------ Time : 10-------------------
loss :  tensor(2.3033, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[771/468] LOSS : 1778.049------ Time : 10-------------------
loss :  tensor(2.2954, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[772/468] LOSS : 1780.345------ Time : 10-------------------
loss :  tensor(2.3038, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[773/468] LOSS : 1782.648------ Time : 10-------------------
loss :  tensor(2.3005, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[774/468] LOSS : 1784.949------ Time : 10-------------------
loss :  tensor(2.2944, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[775/468] LOSS : 1787.243------ Time : 10-------------------
loss :  tensor(2.2989, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[776/468] LOSS : 1789.542------ Time : 10-------------------
loss :  tensor(2.3086, device='cud

[1/1]-----[847/468] LOSS : 1953.139------ Time : 11-------------------
loss :  tensor(2.2999, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[848/468] LOSS : 1955.439------ Time : 11-------------------
loss :  tensor(2.3367, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[849/468] LOSS : 1957.776------ Time : 11-------------------
loss :  tensor(2.2702, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[850/468] LOSS : 1960.046------ Time : 11-------------------
loss :  tensor(2.3153, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[851/468] LOSS : 1962.361------ Time : 11-------------------
loss :  tensor(2.2799, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[852/468] LOSS : 1964.641------ Time : 11-------------------
loss :  tensor(2.3238, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[853/468] LOSS : 1966.965------ Time : 11-------------------
loss :  tensor(2.3261, device='cuda:0', grad_fn=<NllLossBackward>)
[1/1]-----[854/468] LOSS : 1969.29

KeyboardInterrupt: 